# Wikipedia Scraping

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import seaborn as sns
import re
%matplotlib inline

In [58]:
train = pd.read_csv('../data/train_team.csv')
test = pd.read_csv('../data/test_team.csv')

In [59]:
train.head()

,game_date,home_team,away_team,country,neutral,home_win,attack_diff,bup_dribbling_diff,bup_passing_diff,bup_speed_diff,...,last_year_avg_weighted_diff,last_year_avg_diff,previous_points_diff,rank_change_diff,rank_diff,three_year_ago_avg_diff,three_year_ago_weighted_diff,total_points_diff,two_year_ago_avg_diff,two_year_ago_weighted_diff
0,2010-03-03 00:00:00+00:00,Mexico,New Zealand,USA,True,1,13.0,-0.0,-25.0,-5.0,...,-0.0,-0.0,531.0,-3.0,-62.0,-0.0,-0.0,-0.0,-0.0,-0.0
1,2010-05-24 00:00:00+00:00,Australia,New Zealand,Australia,False,1,11.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,489.0,-2.0,-58.0,-0.0,-0.0,-0.0,-0.0,-0.0
2,2010-06-20 00:00:00+00:00,Italy,New Zealand,South Africa,True,2,23.0,-0.0,-40.0,-3.0,...,-0.0,-0.0,771.0,-0.0,-73.0,-0.0,-0.0,-0.0,-0.0,-0.0
3,2010-06-04 00:00:00+00:00,Slovenia,New Zealand,Slovenia,False,1,10.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,447.0,-2.0,-53.0,-0.0,-0.0,-0.0,-0.0,-0.0
4,2010-06-24 00:00:00+00:00,Paraguay,New Zealand,South Africa,True,2,15.0,-0.0,-30.0,-5.0,...,-0.0,-0.0,409.0,-1.0,-47.0,-0.0,-0.0,-0.0,-0.0,-0.0


In [60]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import unicodedata
import re

def get_countries_by_continent():    
    url = "https://simple.wikipedia.org/wiki/List_of_countries_by_continents"
    my_page = requests.get(url)
    soup = BeautifulSoup(my_page.content, "lxml")
    continents_obj = soup.findAll('span',{'class':'mw-headline'})
    continents_obj = continents_obj[:-2]
    country_to_continent = {}
    
    continents = []
    for continent in continents_obj:
        continents.append(continent.text)
        raw_countries = continent.findNext('ul')
        raw_countries = raw_countries.findAll('li')
        for c in raw_countries:
            country_name = c.text.split("-")[0]
            # remove secondary names
            idx = country_name.find('(')
            idx2 = country_name.find('[')
            if idx != -1:
                country_name = country_name[:idx]
            if idx != -2:
                country_name = country_name[:idx2].strip()
            country_name = country_name.replace('*','')
            country_to_continent[country_name] = continent.text
            
    
    return country_to_continent,continents

In [61]:
country_to_continent,continents = get_countries_by_continent()

In [62]:
continents

['Africa',
 'Antarctica',
 'Asia',
 'Europe',
 'America',
 'North America',
 'Central America and the Antilles',
 'South America',
 'Oceania',
 'Australia',
 'Australasian']

In [63]:
country_to_continent

{'Algeria': 'Africa',
 'Angola': 'Africa',
 'Benin': 'Africa',
 'Botswana': 'Africa',
 'Burkina Faso': 'Africa',
 'Burundi': 'Africa',
 'Cameroon': 'Africa',
 'Cape Verde': 'Africa',
 'Central African Republic': 'Africa',
 'Chad': 'Africa',
 'Comoros': 'Africa',
 'Republic of the Congo': 'Africa',
 'Democratic Republic of the Congo': 'Africa',
 "Côte d'Ivoire": 'Africa',
 'Djibouti': 'Africa',
 'Egypt': 'Africa',
 'Equatorial Guinea': 'Africa',
 'Eritrea': 'Africa',
 'Ethiopia': 'Africa',
 'Gabon': 'Africa',
 'The Gambia': 'Africa',
 'Ghana': 'Africa',
 'Guinea': 'Africa',
 'Guine': 'Africa',
 'Kenya': 'Africa',
 'Lesotho': 'Africa',
 'Liberia': 'Africa',
 'Libya': 'Africa',
 'Madagascar': 'Africa',
 'Malawi': 'Africa',
 'Mali': 'Africa',
 'Mauritania': 'Africa',
 'Mauritius': 'Africa',
 'Morocco': 'Africa',
 'Mozambique': 'Africa',
 'Namibia': 'Africa',
 'Niger': 'Africa',
 'Nigeria': 'Africa',
 'Rwanda': 'Africa',
 'São Tomé and Príncipe': 'Africa',
 'Senegal': 'Africa',
 'Seychelles

In [64]:
country_to_continent['Ivory Coast'] = country_to_continent.pop("Côte d'Ivoire")
country_to_continent['England'] = country_to_continent.pop("United Kingdom")
country_to_continent['Korea Republic'] = country_to_continent.pop("South Korea")
country_to_continent['USA'] = country_to_continent.pop("United States of America")
country_to_continent['Ireland'] = country_to_continent.pop("Republic of Ireland")
country_to_continent['Northern Ireland'] = country_to_continent['Ireland']
country_to_continent['Iceland'] = country_to_continent.pop("Eastern Iceland")
country_to_continent['Wales'] = country_to_continent['England']
country_to_continent['Scotland'] = country_to_continent['England']

In [65]:
#Get set of countries
my_set = set()
for k,_ in country_to_continent.items():
    my_set.add(k)

In [66]:
my_set

{'Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australi',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bonaire',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'Czech Republic',
 'Democratic Republic of the Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'East Timor',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'England',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Federated States of Micronesia',
 'Fiji',
 'Finland',
 'Flore',
 'France',
 'French Guiana',
 'Gabon',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 '

In [67]:
#Ensure all countries properly mapped
for c in train.country.unique():
    if c not in my_set:
        print(c)
        
for c in train.home_team.unique():
    if c not in my_set:
        print(c)

for c in train.away_team.unique():
    if c not in my_set:
        print(c)

Réunion


In [68]:
countries = {}
for k,v in country_to_continent.items():
    if k in train.home_team.unique():
         countries[k] = v

In [69]:
def scrape_gdp_pp():
    url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(PPP)_per_capita"
    my_page = requests.get(url)
    soup = BeautifulSoup(my_page.content, "lxml")
    tables = soup.find_all('table',{'class':'wikitable sortable'})
    country_rows = tables[0].findAll('tr')
    countries_data = {}

    for country in country_rows[1:]:    
        country_data = country.findAll('td')
        name = country_data[1].text
        country_name = unicodedata.normalize("NFKD", name)
        country_name = country_name.strip()
        gdp_pp = int(country_data[2].text.strip().replace(',',''))

        countries_data[country_name] = gdp_pp

    return countries_data

In [70]:
gdp_countries = scrape_gdp_pp()
gdp_countries['Ivory Coast'] = gdp_countries.pop("Côte d'Ivoire")
gdp_countries['England'] = gdp_countries.pop("United Kingdom")
gdp_countries['Korea Republic'] = gdp_countries.pop("South Korea")
gdp_countries['USA'] = gdp_countries.pop("United States")
gdp_countries['Northern Ireland'] = gdp_countries['Ireland']
gdp_countries['Wales'] = gdp_countries['England']
gdp_countries['Scotland'] = gdp_countries['England']


In [71]:
countryset = set(gdp_countries.keys())
for k,v in countries.items():
    if k not in countryset:
        print(k)

In [72]:
for country,_ in gdp_countries.items():
    if country not in country_to_continent:
        print(country)

Macau
Hong Kong
Bahamas, The
World[n 1]
Congo, Rep.
Timor-Leste
Micronesia
São Tomé and Príncipe
Guinea-Bissau
Gambia, The
Congo, Dem. Rep.


In [92]:
gdp_col = []
for col1,col2 in zip(train['home_team'].items(), train['away_team'].items()):
    team1 = col1[1]
    team2 = col2[1]   
    if country_to_continent[team1] != country_to_continent[team2]:
        gdp_col.append(0)
    else:
        gdp_col.append(float(gdp_countries[team1])/gdp_countries[team2])
train['gdp_diff'] = gdp_col

In [93]:
train.head()

,game_date,home_team,away_team,country,neutral,home_win,attack_diff,bup_dribbling_diff,bup_passing_diff,bup_speed_diff,...,last_year_avg_diff,previous_points_diff,rank_change_diff,rank_diff,three_year_ago_avg_diff,three_year_ago_weighted_diff,total_points_diff,two_year_ago_avg_diff,two_year_ago_weighted_diff,gdp_diff
0,2010-03-03 00:00:00+00:00,Mexico,New Zealand,USA,True,1,13.0,-0.0,-25.0,-5.0,...,-0.0,531.0,-3.0,-62.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.000000
1,2010-05-24 00:00:00+00:00,Australia,New Zealand,Australia,False,1,11.0,-0.0,-0.0,-0.0,...,-0.0,489.0,-2.0,-58.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.295569
2,2010-06-20 00:00:00+00:00,Italy,New Zealand,South Africa,True,2,23.0,-0.0,-40.0,-3.0,...,-0.0,771.0,-0.0,-73.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.000000
3,2010-06-04 00:00:00+00:00,Slovenia,New Zealand,Slovenia,False,1,10.0,-0.0,-0.0,-0.0,...,-0.0,447.0,-2.0,-53.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.000000
4,2010-06-24 00:00:00+00:00,Paraguay,New Zealand,South Africa,True,2,15.0,-0.0,-30.0,-5.0,...,-0.0,409.0,-1.0,-47.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.000000


In [94]:
gdp_col = []
for col1,col2 in zip(test['home_team'].items(), test['away_team'].items()):
    team1 = col1[1]
    team2 = col2[1]   
    if country_to_continent[team1] != country_to_continent[team2]:
        gdp_col.append(0)
    else:
        gdp_col.append(float(gdp_countries[team1])/gdp_countries[team2])
test['gdp_diff'] = gdp_col

In [95]:
test.head()

,game_date,home_team,away_team,country,neutral,home_win,attack_diff,bup_dribbling_diff,bup_passing_diff,bup_speed_diff,...,previous_points_diff,rank_change_diff,rank_diff,three_year_ago_avg_diff,three_year_ago_weighted_diff,total_points_diff,two_year_ago_avg_diff,two_year_ago_weighted_diff,Group,gdp_diff
0,NaN,Russia,Saudi Arabia,Russia,False,1,9.0,9.0,-17.0,-19.0,...,31.0,-4.0,3.0,233.41,46.68,-8.75,-18.66,-5.59,A,0.000000
1,NaN,Egypt,Uruguay,Russia,True,0,-14.0,-8.0,13.0,14.0,...,-340.0,-2.0,31.0,-346.08,-69.21,-368.98,-229.85,-68.96,A,0.000000
2,NaN,Morocco,Iran,Russia,True,0,-7.0,-15.0,-31.0,-31.0,...,-46.0,2.0,4.0,-227.79,-45.55,-22.49,58.77,17.63,B,0.000000
3,NaN,Portugal,Spain,Russia,True,2,-0.0,27.0,22.0,19.0,...,144.0,2.0,-6.0,183.64,36.73,148.19,-128.63,-38.59,B,0.792696
4,NaN,France,Australia,Russia,True,1,16.0,6.0,-19.0,-14.0,...,466.0,-4.0,-29.0,358.74,71.75,479.80,66.44,19.93,C,0.000000


In [96]:
train.to_csv('../data/train_merged.csv', index = False)
test.to_csv('../data/test_merged.csv', index = False)